# Assignment 9, GR10NR9

This assignment calculates the hydrogen concentration in aluminium melt as a function of time. The change in hydrogen concentration during a small intervall dt is described by the realationship:

$$
\begin{align*}
\frac{d[\%H]}{dt}=-\frac{100 M_H}{M}\left[ \frac{2 [\%H]^2 f_H^2 K_H^2 G_m Z^2}{p_{in}} + \frac{k_{ts} A_s \rho ([\%H]-[\%H]_v)}{100 M_H}\right]\tag{1}
\label{eq1}
\end{align*}
$$

where everything but $[\%H]$ are constants, and $Z$ is given by:

$$
\begin{align*}
-Z -ln(1-Z)= \frac{\rho k_t A p_{in}}{400 M_H G_m f_H^2 [\%H] K_H^2}\tag{2}
\label{eq2}
\end{align*}
$$

In order to find Z, two different methods were used; Newton's method and and a python function called scipy.opyimize.fsolve. The last method is referenced as Solver in the rest of the assignment. 

$A$ is the total surface area of the bubbles and given by the relationship:

$$
\begin{align*}
A = \frac{6 G_v \tau}{d_b} \tag{3}
\label{eq3}
\end{align*}
$$

where $\tau$ is the average time the bubbles stay in the melt and is given by:

$$
\begin{align*}
\tau = \frac{h}{0.32\left(\frac{0.004}{d_b} \right)^{1/8}}\tag{4}
\label{eq4}
\end{align*}
$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import *
from IPython.display import clear_output
import pandas as pd

newparams = {'figure.figsize': (15, 9), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral', 'figure.dpi': 200}
plt.rcParams.update(newparams)

In [2]:
#constants
M = 200 #kg
K_H  = 0.78*10**4 #atm**(0.5)
k_ts = 2.3*10**(-3) #m/s
kt = 2.3*10**(-4) #m/s
h = 0.5 #m
M_H = 1 #kg/kmol
G_m = 4.5*10**(-6) #kmol/s
A_s = 0.17 #m**2
G_v = 22.4*G_m #m**3/s
f_H = 1.75
p_in = 1.0 #atm
rho = 2350 #kg/m**3
d_b = 0.004 #m

### Calculating Z using Newton's method and/or fsolver

In [3]:
tau = h/(0.32*(0.004/d_b)**(1/8))

A = (6*G_v*tau)/d_b

In [4]:
def funkZ(H, z):
    K = (rho*kt*A*p_in)/(400*M_H*G_m*(f_H)**2*H*(K_H)**2)
    Z = z+np.log(1-z)+K
    return Z

In [5]:
#Calculating Z by Newton's method:

def dfZ(Z):
    return 1-1/(1-Z)

def newton(H):
    Z0 = 0.5
    tol = 10**(-4)
    df = dfZ(Z0)
    max_iter=30
    
    fZ = funkZ(H, Z0)
    while abs(fZ) > tol:           # Accept the solution 
        Z0 = Z0 - fZ/dfZ(Z0)            # Newton-iteration
        fZ = funkZ(H,Z0)
    return Z0

In [6]:
#Calculating Z by function scipy.optimize.fsolve:

def Zsolve(Z0, H): #Mått lag en funksjon som tar inn funkZ ellers va fsolver = 0.5 hele tida.
    x = funkZ(H, Z0)
    return x

def solver(H): #regner ut nye Z verdien
    Z = fsolve(Zsolve, 0.5, args=H)
    return Z


In [7]:
#calculating dH/dt

t = np.arange(0, 1010, 10) #want the time from 1-1000 seconds with 10s step.


def dhdt(H, HV, valg):
    if valg == 1: #newton
        Z = newton(H)
        dHdt1 = - (100* M_H/M)*((2*H**2*f_H**2*K_H**2*G_m*Z**2/p_in)+(k_ts*A_s*rho*((H-HV)/(100*M_H))))
    else:
        Z = solver(H)
        dHdt1 = -(100* M_H/M)*((2*H**2*f_H**2*K_H**2*G_m*Z**2/p_in)+(k_ts*A_s*rho*((H-HV)/(100*M_H))))                                                                 
    return dHdt1

def rungekutta(H, HV, valg):
    h = 10
    y0 = H #er dette fordi y0 skal være den første verdien?
    Hny = np.empty(0)
    Hny = np.append(Hny, y0)  
    
    for i in range(len(t)-1):
        
        #making the RK4 steps:
        
        F1 = h * dhdt(y0, HV, valg)
        F2 = h * dhdt(y0 + 1/2 * F1, HV, valg)
        F3 = h * dhdt(y0 + 1/2 * F2, HV, valg)
        F4 = h * dhdt(y0 + F3, HV, valg)
        
        #h+= 
        y0 = y0 + 1/6* (F1 + 2*F2 + 2*F3 + F4)
        Hny = np.append(Hny, y0)
    
    return Hny

### Z vs f(Z)

In [8]:
def ZfZ(H, HV, valg):
    Z = np.linspace(0, 0.99, 100)
    Hverdier = rungekutta(H, HV, valg)
    fZliste = np.empty(0)
    plotH = np.empty(0)
    
    for h in range(0, len(Hverdier), 10):
        hverdi = Hverdier[h]
        plotH = np.append(plotH, hverdi)
        
        for i in range(len(Z)):
            fZ = funkZ(hverdi, Z[i])
            fZliste = np.append(fZliste, fZ)
            
    fZliste = np.split(fZliste, 11)
    return Z, fZliste, plotH

### Mean concentration


In [13]:
def mean(Hliste):
    
    Hmean = np.empty(0)
    for i in range(10, len(Hliste), 10):
        Hsum = np.sum(Hliste[:i])
        mean = (1/i)*Hsum
        Hmean = np.append(Hmean, mean)
    return Hmean

### Plots

In [35]:
def plotZfZ(H, HV, valg):
    Z, fZliste, plotH = ZfZ(H*10**(-6), HV, valg)
    colors = ['red', 'peachpuff', 'orange', 'chartreuse', 'limegreen', 'turquoise', 'cyan', 'blue', 'hotpink', 'crimson']
    col = 0
    for i in range(len(plotH)-1):
        plt.plot(Z, fZliste[i], label="%H="+ str(np.round(plotH[i],7)), color=colors[col])
        col += 1

    for i in range(len(plotH)-1):
        z=newton(plotH[i])
        plt.scatter(z,0, color = 'grey')
        plt.vlines(z, -1, 0, colors='grey',linestyle = '--' )
    plt.axhline(y = 0, color = 'grey', linestyle = '--', label = "f(Z) = 0")
    plt.legend()
    plt.title('f(Z) vs. Z')
    plt.xlabel('Z')
    plt.ylabel('f(Z)')
    plt.grid()
    plt.xlim([0, 1])
    plt.ylim([-1, 0.75])
    plt.show()


def plotHt(H, HV, valg, color1):
    Hliste = rungekutta(H*10**(-6), HV, valg)
    plt.plot(t, Hliste, label = "H(t)", color = color1)
    plt.axhline(y = HV, color = "grey", linestyle = '--', label = "H_v")
    plt.legend()
    plt.title('H(t)')
    plt.xlabel('t')
    plt.ylabel('H[%]')
    plt.grid()
    
    #Table for mean consentrations
    tidny = np.empty(0)
    Hmean = np.round(mean(Hliste), 10)
    for i in range(10, len(t), 10):
        tidny = np.append(tidny, t[i])
    df = pd.DataFrame({
        'Time, [s]': tidny,
        'Mean concentration, H[%]': Hmean,
        })
    pd.options.display.precision = 10
    display(df)
    

In [36]:
#def table(H, HV, valg):
#    Hliste = rungekutta(H, HV, valg)
    

#print(table(15*10**(-6), 1*10**(-6), 1))

### Menu

In [39]:
valg_knapp = widgets.Dropdown(
    options=[('Newton',1),('Solver',2)],
    value = 1,
    description='Z-method:',
    disabled=False
) 

Hslider = widgets.FloatSlider(value = 18, min = 15, max = 20, step = 1, description='H[%]*10^(-6)')

HVknapp = widgets.ToggleButtons(options={'0.5*10**(-6)': 0.5*10**(-6), '1*10**(-6)':1*10**(-6), '1.5*10**(-6)': 1.5*10**(-6)}, value = 1*10**(-6), description='H_v[%]:',)

color_picker = widgets.ColorPicker(description='Pick a color', value= "purple")

def sub_task_chooser(sub):
    
    # Switch function
    if sub == 1: #plotting for H(t)
        widgets.interact(plotHt, H=Hslider, HV=HVknapp, valg = valg_knapp, color1 = color_picker)
        
            

    elif sub == 2: #plotting for f(Z) vs. Z
        widgets.interact(plotZfZ, H=Hslider, HV=HVknapp, valg = valg_knapp)
        

# Menu toggle buttons
sub_menu = widgets.ToggleButtons(options={'H(t)':1, 'f(Z) vs Z':2}, value = 1, description='Plot:', button_style='primary',)
#red buttons

interact(sub_task_chooser, sub=sub_menu)

interactive(children=(ToggleButtons(button_style='primary', description='Plot:', options={'H(t)': 1, 'f(Z) vs …

<function __main__.sub_task_chooser(sub)>